## Segmentação dos dados

As características normalmente são extraídas sobre pequenos segmentos de tamanho fixo dos dados, não no dado como um todo. Estes pequenos segmentos são chamados de janelas. A técnica de separar os dados em janelas, recebe o nome de janela deslizante (*sliding window*) e é uma forma de segmentação de dados. Uma boa prática, é definir um passo para essa janela de forma que haja uma sopreposição de dados, para que informações da lacuna de uma janela e outra não sejam perdidas. Na imagem a seguir podemos observar um sinal EOG. Abaixo dele a representação de janelas: *W1*, *W2*, *W3*... Repare que entre as janelas, há uma sobreposição de tamanho *T*. 

Na base de dados que iremos implementar, cada ensaio tem duração de 5 segundos. Se utilizadas janelas com tamanho de 250 ms, resultará em 20 janelas. Ao aplicar uma sobreposição de ~128ms, ficamos com 41 janelas de ~122ms.

Algumas características de EMG trabalham com o dado no domínio da frequência. Quando tais características são aplicadas, é necessário tranformar o dado para o domínio da frequência, utilizando o método de transformação de domínio `STFT`, do inglês *Short-time Fourier transform* (Transformada de Fourier de curto termo). O código divide o dado em segmentos, tanto no domínio do tempo quanto no domínio da frequência.

In [2]:
# Criando dados fictícios

import numpy as np

shape = (60, 4, 1000)
data = np.random.normal(loc=0, scale=10, size=shape).astype(np.float32)
data

array([[[  9.715339  ,  15.959176  ,  16.09355   , ..., -14.468535  ,
          22.011942  ,  -2.2860115 ],
        [  1.083166  ,   4.8331575 ,  -3.8758605 , ...,   7.46218   ,
           6.7929797 ,   3.5564976 ],
        [-16.891994  ,  -1.0153096 ,  21.34562   , ...,   6.7818613 ,
          19.03512   ,  20.873432  ],
        [ 15.34505   ,   9.304999  ,  -0.73898715, ...,  22.92291   ,
           1.3304133 ,   3.2307098 ]],

       [[  9.850015  , -13.348989  ,   1.592202  , ...,  20.787218  ,
           5.7125893 ,  -5.8822246 ],
        [  4.692102  , -15.622506  , -19.8213    , ...,  -5.0602646 ,
          -7.136052  ,  30.987583  ],
        [  4.0775924 ,  -1.9215442 ,   4.3385696 , ...,   9.433404  ,
           5.5496564 ,  -6.9395437 ],
        [ 10.467308  , -10.872599  ,   0.91100365, ...,  -4.573705  ,
           4.071464  ,  -4.3732624 ]],

       [[ 15.481944  ,   7.8538375 ,  -8.373309  , ...,  -4.678457  ,
          -0.60716796,  -0.66716206],
        [ -8.829713  , -

In [3]:
from scipy.signal import stft

step = 29
segment = 64
print('', data.shape)

n_win = int((data.shape[-1] - segment) / step) + 1
ids = np.arange(n_win) * step

# Janelas do dado no dominio do tempo
chunks_time = np.array([data[:,:,k:(k + segment)] for k in ids]).transpose(1, 2, 0, 3)

# Janelas do dado no domínio da frequência
_, _, chunks_freq = stft(data, fs=200, nperseg=64, noverlap=32)
chunks_freq = np.swapaxes(chunks_freq, 2, 3)

print('Formato (shape) dos dados depois da divisão de janelas')
print(f'Dominio do tempo: {chunks_time.shape} - (classes+ensaios, canais, janelas, linhas)')
print(f'Dominio da frequência:  {chunks_freq.shape} - (classes+ensaios, canais, janelas, linhas)')

 (60, 4, 1000)
Formato (shape) dos dados depois da divisão de janelas
Dominio do tempo: (60, 4, 33, 64) - (classes+ensaios, canais, janelas, linhas)
Dominio da frequência:  (60, 4, 33, 33) - (classes+ensaios, canais, janelas, linhas)


*Desafio*: Implementar um algoritmo para encontrar a mesma quantidade de janelas em ambos os domínios (tempo e frequência).

In [4]:
# Teste Desafio
segment = 64
nperseg_freq = 64
print('', data.shape)

overlap_time = 0  # Iniciando com sobreposição zero no domínio do tempo
overlap_freq = nperseg_freq // 2  # Sobreposição de 50% no domínio da frequência

while True:
    # Cálculo de n_win e criação de ids
    n_win = int((data.shape[-1] - segment) / (overlap_time + 1)) + 1
    ids = np.arange(n_win) * (overlap_time + 1)

    # Janelas no domínio do tempo
    chunks_time = [data[:, :, k:(k + segment)] for k in ids]
    chunks_time = np.array(chunks_time).transpose(1, 2, 0, 3)

    # Janelas no domínio da frequência
    _, _, chunks_freq = stft(data, fs=200, nperseg=nperseg_freq, noverlap=overlap_freq)
    chunks_freq = np.swapaxes(chunks_freq, 2, 3)

    # Verificando se o número de janelas é o mesmo
    if chunks_time.shape[2] == chunks_freq.shape[2]:
        break  # Se o número de janelas for igual, saia do loop
    else:
        overlap_time += 1  # Incrementa a sobreposição no domínio do tempo para tentar igualar o número de janelas

print('Formato (shape) dos dados depois da divisão de janelas')
print(f'Dominio do tempo: {chunks_time.shape} - (classes+ensaios, canais, janelas, linhas)')
print(f'Dominio da frequência:  {chunks_freq.shape} - (classes+ensaios, canais, janelas, linhas)')

 (60, 4, 1000)
Formato (shape) dos dados depois da divisão de janelas
Dominio do tempo: (60, 4, 33, 64) - (classes+ensaios, canais, janelas, linhas)
Dominio da frequência:  (60, 4, 33, 33) - (classes+ensaios, canais, janelas, linhas)
